In [2]:
!pip install librosa numpy pandas scikit-learn tensorflow

In [9]:
import zipfile

zip_path = "/content/archive (1).zip"
extract_path = "/content/ravdess"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Done ✅")

Extraction Done ✅


In [10]:
import os

os.listdir("/content/ravdess")

['Actor_04',
 'Actor_02',
 'Actor_24',
 'Actor_05',
 'Actor_07',
 'Actor_22',
 'Actor_11',
 'Actor_13',
 'Actor_03',
 'Actor_23',
 'Actor_10',
 'Actor_17',
 'Actor_08',
 'Actor_01',
 'Actor_18',
 'Actor_15',
 'Actor_16',
 'audio_speech_actors_01-24',
 'Actor_19',
 'Actor_12',
 'Actor_20',
 'Actor_21',
 'Actor_06',
 'Actor_14',
 'Actor_09']

In [11]:
import os
import librosa
import numpy as np
import pandas as pd

In [12]:
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "05": "angry",
    "06": "fear"
}

In [13]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, duration=3, offset=0.5)

    mfcc = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)

    return np.hstack([mfcc, chroma, mel])

In [14]:
X = []
y = []

data_path = "/content/ravdess"

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".wav"):
            emotion_code = file.split("-")[2]

            if emotion_code in emotion_map:
                label = emotion_map[emotion_code]

                # Convert to Distress / Normal
                if label in ["angry", "fear"]:
                    y.append(1)   # Distress
                else:
                    y.append(0)   # Normal

                file_path = os.path.join(root, file)
                features = extract_features(file_path)
                X.append(features)

X = np.array(X)
y = np.array(y)

print("Dataset Shape:", X.shape)
print("Labels Shape:", y.shape)

Dataset Shape: (1344, 180)
Labels Shape: (1344,)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Training Samples:", X_train.shape)
print("Testing Samples:", X_test.shape)

Training Samples: (1075, 180)
Testing Samples: (269, 180)


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        46,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,361 (310.00 KB)

 Trainable params: 79,361 (310.00 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7617 - loss: 0.4733 - val_accuracy: 0.9182 - val_loss: 0.2251
Epoch 2/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8793 - loss: 0.2835 - val_accuracy: 0.9554 - val_loss: 0.1549
Epoch 3/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9197 - loss: 0.1914 - val_accuracy: 0.9517 - val_loss: 0.1230
Epoch 4/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9320 - loss: 0.1628 - val_accuracy: 0.9554 - val_loss: 0.0984
Epoch 5/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9451 - loss: 0.1408 - val_accuracy: 0.9814 - val_loss: 0.0811
Epoch 6/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9621 - loss: 0.1097 - val_accuracy: 0.9703 - val_loss: 0.0712
Epoch 7/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9731 - loss: 0.0789 - val_accuracy: 0.9740 - val_loss: 0.0703
Epoch 8/40
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9842 - loss: 0.0643 - val_accuracy: 0.9851 - val_l

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Final Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0033 
Final Test Accuracy: 1.0


In [20]:
model.save("distress_model.h5")

In [21]:
import pickle

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [26]:
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model("distress_model.h5")

# Load scaler
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

def predict_distress(file_path):
    features = extract_features(file_path)
    features = features.reshape(1, -1)
    features = scaler.transform(features)

    prediction = model.predict(features)[0][0]

    print("Prediction Score:", prediction)

    if prediction > 0.75:
        print("🚨 HIGH CONFIDENCE DISTRESS")
    elif prediction > 0.5:
        print("⚠ POSSIBLE DISTRESS")
    else:
        print("✅ NORMAL VOICE")

# TEST HERE
predict_distress("/content/sample voice 2 .wav")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
Prediction Score: 0.0
✅ NORMAL VOICE
